In [1]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tldextract
import re

### Load Data

In [10]:
toxic_df = pd.read_csv("toxic.tsv",sep='\t')

In [2]:
tsv_df = pd.read_csv("../data/val.tsv",sep='\t')

In [2]:
train_file = '../data/train.jsonl'
validation_file = '../data/validation.jsonl'
train_df = pd.read_json(train_file, lines = True)
validation_df = pd.read_json(validation_file, lines = True)

train_df = train_df.append(validation_df)

C:\Users\Vi\AppData\Local\Temp\ipykernel_17544\1950742278.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(validation_df)


In [3]:
## Transforming Data Step 1:
train_df = train_df.drop(['postId', 'targetMedia', 'provenance', 'spoiler', 'spoilerPositions', 'targetTitle'], axis=1)
#validation_df = validation_df.drop(['postId', 'targetMedia', 'provenance', 'targetDescription', 'targetTitle'], axis=1)

train_df['targetUrl'] = train_df['targetUrl'].astype(str)
#validation_df['targetUrl'] = validation_df['targetUrl'].astype(str)
train_df['targetUrl'] = train_df.targetUrl.apply(lambda x: '.'.join(tldextract.extract(x)[1:]))
#validation_df['targetUrl'] = validation_df.targetUrl.apply(lambda x: '.'.join(tldextract.extract(x)[1:]))

train_df = train_df.fillna("")
#validation_df = validation_df.fillna("")
train_df['targetDescription'] = train_df["targetDescription"].astype(str)
train_df['postText'] = train_df["postText"].astype(str)

In [4]:
## Calculated Measures
train_df["title_spoiler_ratio"] = train_df.apply(lambda x: len(x["targetDescription"].split(" ")) / len(x["postText"].split(" ")), axis=1 )
#train_df["first_spoiler"] = train_df.apply(lambda x: x["spoilerPositions"][0][0][0], axis=1)

string_list = []
for i, j in train_df.iterrows():
    string = ""
    for k in j["targetParagraphs"]:
        string += k + " "
    string_list.append(string)
train_df["full_context"] = string_list


In [5]:
## Enrich with NER data
ner_df  = pd.read_json("../data/ner/train_ner.json")
ner2_df = pd.read_json("../data/ner/validation_ner.json")

ner2_df = ner2_df.drop(['postText', 'postPlatform', "targetParagraphs", "targetTitle", "targetDescription", "targetKeywords", "targetMedia", "targetUrl", "provenance", "spoiler", "spoilerPositions", "tags", "full_context"], axis=1)
ner_df =  ner_df.append(ner2_df, ignore_index=True)

train_df = pd.merge(train_df, ner_df, on='uuid', how='left')

C:\Users\Vi\AppData\Local\Temp\ipykernel_17544\1617025903.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ner_df =  ner_df.append(ner2_df, ignore_index=True)


In [7]:
ner_df.tail()

,uuid,postId,ner_orgs,ner_persons,ner_dates,ner_locations
3995,1189d343-42eb-47e7-8395-ff978a683875,428006164904034305,[YouTube],"[Kyle, Josh]",[this week],[]
3996,7912282b-137b-4098-875d-8ad9f19354a8,806153730206892032,"[The New York Times, Politico, Harvard Univers...","[Suprun, Donald Trump, George W. Bush, Christo...","[each day, Sept. 11, 9/11, Dec. 19, two days]","[Texas, Ohio, America]"
3997,1fdf71e8-ec14-4c3b-a7c5-ca678c6f8ccb,847331053991813120,[Instagram],"[Instagram, Rachel Crawley ☾, Crawley, nomakeu...","[22-year-old, 2017, years, today, Mar 3, 2017,...",[Crawley]
3998,17f6b540-cf8d-4ddf-8321-1c9ce2315d71,788056531304583168,"[Reddit, CBS, Assange, Equador, Gizmodo, CNET,...","[Declan McCullagh, John Kerry, Assange, Roger ...","[June 2012, 16 October 2016, October 17, 2016,...","[UK, Ecuador, London, U.S.]"
3999,89dcad77-d8ad-4705-8676-717b26fda2ad,388308677494444032,"[Munch, HARGITAY, The Huffington Post, SVU, NBC]","[Belzer, Finn Wittrock, Matt DeCapua, John Mun...","[more than 20 years, Oct. 9, 1993, May 2013, 1...",[]


### Combine into Text

In [12]:
def combine_features(data_df):
    all_features = []
    labels = []

    for index, row in data_df.iterrows():
        """combined = ''
        combined += f'The post contains the title {row["postText"][1:-1]}. ' \
                    f'The spoiler has a length ratio of {row["title_spoiler_ratio"]}. '
                    #f'The post contains the spoiler {row["spoiler"][1:-1]}. ' \
                    #f'The spoiler first appears at paragraph {row["first_spoiler"]}. ' \

        if re.match(".*\d+\s*[\.\)].+\d+?\s*[\.\)].+?\d+\s*[\.\)]",row["full_context"], re.MULTILINE | re.IGNORECASE):
            combined += f'The spoiler contains an enumeration or multi-line. '
        
        if len(row["ner_orgs"]) > 0:
            combined += f'The context involves {len(row["ner_orgs"])} organisations. '
            #combined += f'The context contains the following organisations {str(row["ner_orgs"])[1:-1]}. '
        if len(row["ner_persons"]) > 0:
            combined += f'The context involves {len(row["ner_persons"])} persons. '
            #combined += f'The context contains the following persons {str(row["ner_persons"])[1:-1]}. '
        if len(row["ner_dates"]) > 0:
            combined += f'The context involves {len(row["ner_dates"])} dates. '
            #combined += f'The context contains the following dates {str(row["ner_dates"])[1:-1]}. '
        if len(row["ner_locations"]) > 0:
            combined += f'The context involves {len(row["ner_locations"])} locations. '
            #combined += f'The context contains the following locations {str(row["ner_locations"])[1:-1]}. '
            
        combined += f'The post was published on {row["postPlatform"]}. ' \
                    f'The post is sourced from the website {row["targetUrl"]}. '
        combined = combined.replace('"', "'")
        all_features.append(combined)
        labels.append(row["tags"][0])"""
        combined = ''
        combined += f'Title: {row["postText"][1:-1]}. ' \
                    f'Spoiler Length Ratio: {row["title_spoiler_ratio"]}. '
        if re.match(".*\d+\s*[\.\)].+\d+?\s*[\.\)].+?\d+\s*[\.\)]",row["full_context"], re.MULTILINE | re.IGNORECASE):
            combined += f'Enumeration or multi-line. '
        
        if len(row["ner_orgs"]) > 0:
            combined += f'{len(row["ner_orgs"])} organisations. '
            #combined += f'The context contains the following organisations {str(row["ner_orgs"])[1:-1]}. '
        if len(row["ner_persons"]) > 0:
            combined += f'{len(row["ner_persons"])} persons. '
            #combined += f'The context contains the following persons {str(row["ner_persons"])[1:-1]}. '
        if len(row["ner_dates"]) > 0:
            combined += f'{len(row["ner_dates"])} dates. '
            #combined += f'The context contains the following dates {str(row["ner_dates"])[1:-1]}. '
        if len(row["ner_locations"]) > 0:
            combined += f'{len(row["ner_locations"])} locations. '
            #combined += f'The context contains the following locations {str(row["ner_locations"])[1:-1]}. '
            
        combined += f'Publishing Platform: {row["postPlatform"]}. ' \
                    f'Source Website {row["targetUrl"]}. '
        combined = combined.replace('"', "'")
        all_features.append(combined)
        labels.append(row["tags"][0])


    return all_features, labels

all_features, labels = combine_features(train_df)

In [13]:
all_features[97]

"Title: 'One thing women would choose over sex that we're not even surprised about'. Spoiler Length Ratio: 2.1538461538461537. 2 persons. 4 dates. Publishing Platform: Twitter. Source Website huff.to. "

In [11]:
display(train_df.iloc[97])

uuid                                4cd4e1f1-7425-4f6e-b520-6335be81724c
postText               ["One thing women would choose over sex that w...
postPlatform                                                     Twitter
targetParagraphs       [Carving out time for yourself during the day ...
targetDescription      Carving out time for yourself during the day -...
targetKeywords         Love & Sex,things women prefer to sex,sex,the ...
targetUrl                                                        huff.to
tags                                                            [phrase]
title_spoiler_ratio                                             2.153846
full_context           Carving out time for yourself during the day -...
postId                                                399413489804275712
ner_orgs                                                              []
ner_persons                     [Celestial Seasonings, Christina Norman]
ner_dates                   [the day, each day, the

In [10]:
max(all_features, key=len)

"The post contains the title 'Daniel Craig Was Offered A Staggering Amount Of Money To Carry On Playing Bond'. The spoiler has a length ratio of 0.07142857142857142. The spoiler contains an enumeration or multi-line. The context involves 13 organisations. The context involves 14 persons. The context involves 9 dates. The context involves 5 locations. The post was published on Facebook. The post is sourced from the website James (Cash) Bond! Daniel Craig is 'offered a staggering $150million by Sony bosses for two more 007 movies'.. "

In [9]:
train_tsv = pd.DataFrame(list(zip(all_features, labels)),
              columns=['text','label'])

In [10]:
train_tsv.to_csv("../data/full_train_short.tsv", sep="\t", encoding='utf-8', index=False)

In [19]:
train_tsv[:3200].to_csv("../data/train.tsv", sep="\t", encoding='utf-8')

In [20]:
train_tsv[3200:].to_csv("../data/val.tsv", sep="\t", encoding='utf-8')

### Data Exploration: 

In [6]:
input_df

,uuid,postId,postText,postPlatform,targetParagraphs,targetTitle,targetDescription,targetKeywords,targetMedia,targetUrl,provenance,spoiler,spoilerPositions,tags
0,0af11f6b-c889-4520-9372-66ba25cb7657,532quh,"[Wes Welker Wanted Dinner With Tom Brady, But ...",reddit,[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,","[http://pixel.wp.com/b.gif?v=noscript, http://...",http://nesn.com/2016/09/wes-welker-wanted-dinn...,"{'source': 'anonymized', 'humanSpoiler': 'They...",[how about that morning we go throw?],"[[[3, 151], [3, 186]]]",[passage]
1,b1a1f63d-8853-4a11-89e8-6b2952a393ec,411701128456593408,[NASA sets date for full recovery of ozone hole],Twitter,[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[http://s.m.huffpost.com/assets/Logo_Huffingto...,http://huff.to/1cH672Z,"{'source': 'anonymized', 'humanSpoiler': '2070...",[2070],"[[[0, 0], [0, 4]]]",[phrase]
2,008b7b19-0445-4e16-8f9e-075b73f80ca4,380537005123190784,[This is what makes employees happy -- and it'...,Twitter,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[http://i.huffpost.com/gen/1359674/images/o-HA...,http://huff.to/1epfeaw,"{'source': 'anonymized', 'humanSpoiler': 'Inte...",[intellectual stimulation],"[[[1, 186], [1, 210]]]",[phrase]
3,31ecf93c-3e21-4c80-949b-aa549a046b93,844567852531286016,[Passion is overrated — 7 work habits you need...,Twitter,"[It’s common wisdom. Near gospel really, and n...","‘Follow your passion’ is wrong, here are 7 hab...",There's a lot more to work that loving your job,"business, work-life, careers",None,None,"{'source': 'anonymized', 'humanSpoiler': None,...",[Purpose connects us to something bigger and i...,"[[[11, 25], [11, 101]], [[17, 56], [17, 85]], ...",[multi]
4,31b108a3-c828-421a-a4b9-cf651e9ac859,814186311573766144,[The perfect way to cook rice so that it's per...,Twitter,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,The question 'How does one cook rice properly?...,"Quora,users,share,perfect,way,cook,rice",None,None,"{'source': 'anonymized', 'humanSpoiler': None,...",[in a rice cooker],"[[[5, 60], [5, 76]]]",[phrase]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,92578045-699f-4957-a3c5-cff2c3874dae,591979258442027008,[Has Facebook's video explosion completely sha...,Twitter,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",.,"Facebook,web video,web video ads,YouTube",[http://djcs-prod.s3.amazonaws.com/wsjfrontpag...,http://on.wsj.com/1GfyUqz,"{'source': 'anonymized', 'humanSpoiler': 'No.'...",[it hasn’t necessarily taken the wind out of Y...,"[[[7, 50], [7, 118]]]",[passage]
3196,51682121-df0b-4289-a95f-e1bc3d181306,881531941974661_902284433232745,[Cop Is Eating At A Chili's When Teen Hands Hi...,Facebook,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,None,None,[https://sbly-web-prod-shareably.netdna-ssl.co...,http://shareably.net/officer-receives-touching...,"{'source': 'anonymized', 'humanSpoiler': 'The ...","[It read, ""Thanks for keeping us safe.""]","[[[0, 317], [0, 355]]]",[passage]
3197,9c45ca67-38c4-47b4-aa0d-48434bae09fc,837356193576333314,[5 popular myths about visible signs of aging ...,Twitter,"[Obama looks decades younger already, but what...",5 popular myths about visible signs of aging t...,Dozens of anti-wrinkle products and creams are...,,None,https://www.businessinsider.in/5-popular-myths...,"{'source': 'anonymized', 'humanSpoiler': None,...",[1. Anti-wrinkle creams will er

In [4]:
input_df.dtypes

uuid                 object
postId               object
postText             object
postPlatform         object
targetParagraphs     object
targetTitle          object
targetDescription    object
targetKeywords       object
targetMedia          object
targetUrl            object
provenance           object
spoiler              object
spoilerPositions     object
tags                 object
dtype: object

In [5]:
input_df['tags'].value_counts()

[phrase]     1367
[passage]    1274
[multi]       559
Name: tags, dtype: int64

In [6]:
input_df['postPlatform'].value_counts()

Twitter     1530
reddit      1150
Facebook     520
Name: postPlatform, dtype: int64

In [7]:
input_df['postPlatform'] = input_df['postPlatform'].astype(str)
input_df['tags'] = input_df['tags'].astype(str)

In [8]:
input_df.groupby(['tags', 'postPlatform']).count()
#input_df.groupby([input_df.tags.apply(tuple), input_df.postPlatform.apply(tuple)]).count()

uuid  postId  postText  targetParagraphs  \
tags        postPlatform                                             
['multi']   Facebook        68      68        68                68   
            Twitter        329     329       329               329   
            reddit         162     162       162               162   
['passage'] Facebook       231     231       231               231   
            Twitter        510     510       510               510   
            reddit         533     533       533               533   
['phrase']  Facebook       221     221       221               221   
            Twitter        691     691       691               691   
            reddit         455     455       455               455   

                          targetTitle  targetDescription  targetKeywords  \
tags        postPlatform                                                   
['multi']   Facebook               68                 52              26   
            Twitter               329                327             319   
            reddit                162                139              55   
['passage'] Facebook              231                199             109   
            Twitter               510                497             473   
            reddit                533                458             218   
['phrase']  Facebook              221                195              90   
            Twitter               691                678             649   
            reddit                455                388             177   

                          targetMedia  targetUrl  provenance  spoiler  \
tags        postPlatform                                                
['multi']   Facebook               68         68          68       68   
            Twitter               111        140         329      329   
            reddit                162        162         162      162   
['passage'] Facebook              231        231         231      231   
            Twitter               317        319         510      510   
            reddit                533        533         533      533   
['phrase']  Facebook              221        221         221      221   
            Twitter               587        588         691      691   
            reddit                455        455         455      455   

                          spoilerPositions  
tags        postPlatform                    
['multi']   Facebook                    68  
            Twitter                    329  
            reddit                     162  
['passage'] Facebook                   231  
            Twitter                    510  
            reddit                     533  
['phrase']  Facebook                   221  
            Twitter                    691  
            reddit                     455

In [2]:
import pandas as pd
import datasets
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import wandb
import os

ModuleNotFoundError: No module named 'datasets'